In [75]:
import geopandas as gpd
import plotly.express as px
import json

In [76]:
# Load the dataset (assuming it's a GeoDataFrame)
file_path = r"C:\Users\franb\Documents\Data_Science\Listing_Investments_PT\data\shapes_freguesias_portugal\Cont_AAD_CAOP2017.shp"  # Update the path
gdf_new = gpd.read_file(file_path)

# Ensure CRS is in WGS84 (EPSG:4326) for correct mapping
gdf_new = gdf_new.to_crs(epsg=4326)

In [77]:
# Load the dataset (assuming it's a GeoDataFrame)
file_path = r"C:\Users\franb\Documents\Data_Science\Listing_Investments_PT\data\shapes_antigas_freguesias_portugal\Cont_AAd_CAOP20120.shp"  # Update the path
gdf_old = gpd.read_file(file_path)

# Ensure CRS is in WGS84 (EPSG:4326) for correct mapping
gdf_old = gdf_old.to_crs(epsg=4326)

In [78]:
# Load the Freguesias Mapping
file_path = r"C:\Users\franb\Documents\Data_Science\Listing_Investments_PT\data\freguesias_old_new_mapping.json"  
with open(file_path, "r", encoding="utf-8") as f:
    freguesias_mapping = json.load(f)


In [79]:
gdf_old = gdf_old[gdf_old['Distrito'] == 'PORTO']
gdf_new = gdf_new[gdf_new['Distrito'] == 'PORTO']

In [80]:
gdf_new = gdf_new[(gdf_new['Concelho'] == 'PORTO') | (gdf_new['Concelho'] == 'MATOSINHOS') | (gdf_new['Concelho'] == 'VILA NOVA DE GAIA')]

In [81]:
gdf_old = gdf_old[(gdf_old['Municipio'] == 'PORTO') | (gdf_old['Municipio'] == 'MATOSINHOS') | (gdf_old['Municipio'] == 'VILA NOVA DE GAIA')]


In [82]:
gdf_old["Freguesias_antigas"] = gdf_old["Freguesia"].map(freguesias_mapping)


In [83]:
# Get the current columns list
cols = list(gdf_old.columns)

# Remove the new column if it exists in the list
cols.remove("Freguesias_antigas")

# Insert it at the fifth position (index 4)
cols.insert(4, "Freguesias_antigas")

# Reorder the DataFrame with the new column order
gdf_old = gdf_old[cols]

In [84]:
gdf_old.head()

,Dicofre,Freguesia,Municipio,Distrito,Freguesias_antigas,TAA,Area_EA_Ha,Area_T_Ha,geometry
2754,130805,LEÇA DA PALMEIRA,MATOSINHOS,PORTO,UNIAO_DAS_FREGUESIAS_DE_MATOSINHOS_E_LECA_DA_P...,ÁREA PRINCIPAL,714.41,714.41,"POLYGON ((-8.71283 41.21525, -8.70597 41.21484..."
2761,130802,GUIFÕES,MATOSINHOS,PORTO,"UNIAO_DAS_FREGUESIAS_DE_CUSTOIAS,_LECA_DO_BALI...",ÁREA PRINCIPAL,368.76,368.76,"POLYGON ((-8.66355 41.21714, -8.6635 41.21713,..."
2777,130801,CUSTÓIAS,MATOSINHOS,PORTO,"UNIAO_DAS_FREGUESIAS_DE_CUSTOIAS,_LECA_DO_BALI...",ÁREA PRINCIPAL,603.45,603.45,"POLYGON ((-8.66067 41.22642, -8.66043 41.22639..."
2791,130808,SANTA CRUZ DO BISPO,MATOSINHOS,PORTO,"UNIAO_DAS_FREGUESIAS_DE_PERAFITA,_LAVRA_E_SANT...",ÁREA PRINCIPAL,366.98,366.98,"POLYGON ((-8.66792 41.23284, -8.66841 41.23232..."
2793,130804,LEÇA DO BALIO,MATOSINHOS,PORTO,"UNIAO_DAS_FREGUESIAS_DE_CUSTOIAS,_LECA_DO_BALI...",ÁREA PRINCIPAL,911.31,911.31,"POLYGON ((-8.64469 41.23641, -8.64456 41.2364,..."


# Save GeoJson 
For further use in files and in Google Maps

In [88]:
# For further use
gdf_old.to_file(r"C:\Users\franb\Documents\Data_Science\Listing_Investments_PT\data\maps_geojsons\gdf_old.gpkg", driver="GPKG")

# For Google Maps

gdf_old_kml = gdf_old.copy()
gdf_old_kml = gdf_old_kml[['geometry', 'Freguesia', 'Freguesias_antigas']]

gdf_old_kml.to_file(r"C:\Users\franb\Documents\Data_Science\Listing_Investments_PT\data\maps_geojsons\gdf_old_google.kml", driver="KML")


In [72]:
# Create a filtered dataset (modify filtering as needed)
filtered_data_map = gdf_old.copy()

# Define the metric to visualize (adjust as needed)
selected_metric = "Area_T_Ha"  # Example: total area, can be any numeric column
selected_wider_zones = gdf_old["Freguesia"].unique()  # Example selection logic

In [73]:
# MAP - Choropleth Visualization
fig_map = px.choropleth_map(
    filtered_data_map,
    geojson=filtered_data_map["geometry"],
    locations=filtered_data_map.index,
    color=selected_metric,
    color_continuous_scale="YlOrRd",
    hover_name="Freguesia",
    hover_data={"Municipio": True, selected_metric: True},
    map_style="carto-positron",
    center={"lat": gdf_old.geometry.centroid.y.mean(), "lon": gdf_old.geometry.centroid.x.mean()},
    zoom=10 if len(selected_wider_zones) > 1 else 12,
    opacity=0.5,
)

# Adjust map size
fig_map.update_layout(
    height=800,  # Adjust height
    width=1000,  # Optional: Adjust width
    margin={"r":0,"t":40,"l":0,"b":0}  # Removes unnecessary margins
)

fig_map.show()


C:\Users\franb\AppData\Local\Temp\ipykernel_35412\3278209297.py:11: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


